## Import

In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


### 1 - Télécharger le fichier alice.txt depuis l’espace de cours. Charger ce fichier en python et afficher son contenu.

In [78]:
alice_text = open("alice.txt", encoding='utf-8').read()

### 2 - Depuis la chaîne de caractères précédemment chargée, générer la liste de caractères uniques présents dans le livre. Écrire une méthode permettant de transformer chaque caratère en entier et inversement (chaque entier en caractère). Ceci est nécessaire car les réseaux de neurones ne travaillent pas sur des caractères mais sur des nombres !

In [79]:
unique_car = set()

for letter in alice_text:
    unique_car.add(letter)

In [80]:
car_to_int = {}
int_to_car = {}

for i, c in enumerate(unique_car):
    car_to_int[c] = i
    int_to_car[i] = c

### 3 - Préparer un dataset contenant des séquences de 50 caractères (paramétrable) de longueur en entrée et le caractère suivant cette séquence en target. 

In [81]:
def create_dataframe(data, decalage):
    X, y = [], []
    for i in range(len(data) - decalage):
        X.append(data[i:i+decalage])
        y.append(car_to_int[data[i+decalage]])
    return np.array(X), np.array(y)

In [82]:
data_X, data_y = create_dataframe(alice_text, 50)

In [83]:
df = pd.DataFrame({'x':data_X.tolist(), 'y':data_y})

In [84]:
df

,x,y
0,﻿The Project Gutenberg eBook of Alice’s Advent...,84
1,The Project Gutenberg eBook of Alice’s Adventu...,40
2,he Project Gutenberg eBook of Alice’s Adventur...,47
3,e Project Gutenberg eBook of Alice’s Adventure...,84
4,Project Gutenberg eBook of Alice’s Adventures...,85
...,...,...
163961,ibe to our email newsletter to hear about new ...,23
163962,be to our email newsletter to hear about new e...,64
163963,e to our email newsletter to hear about new eB...,6
163964,to our email newsletter to hear about new eBo...,27


### 4 - Transformer le dataset en séquence « one hot encodées »

In [ ]:
from sklearn.preprocessing import OneHotEncoder

X_int = np.array([[car_to_int[ch] for ch in seq] for seq in df['x']], dtype=np.int32)
enc = OneHotEncoder()
enc.fit(X_int)


ValueError: Expected a 2-dimensional container but got <class 'pandas.core.series.Series'> instead. Pass a DataFrame containing a single row (i.e. single sample) or a single column (i.e. single feature) instead.

In [68]:
X_int.shape

(163966, 50)

In [71]:
df['x'] = X_int

In [74]:
df['x']

0          9
1          8
2         46
3         36
4         84
          ..
163961    40
163962     1
163963    36
163964    84
163965    65
Name: x, Length: 163966, dtype: int32